In [113]:
import pandas as pd
import numpy as np

from synonymes.synonymes import cnrtl, larousse, synonymo, linternaute 

import warnings
warnings.filterwarnings("ignore")

In [106]:
data = pd.read_parquet('data/tokens_v4.parquet')

In [107]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('french')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tkess\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
    def synonymize_from_dic(word, dic):
        if word.isalpha() and word.lower() not in stopwords:
            synonyms = cnrtl(word)
            n = len(synonyms)
            if n > 0:
                r = np.random.randint(0,n)
                synonym = synonyms[r]
                return synonym
            else:
                return 'No synonym found'
        return 'No synonym found'

In [109]:
def synonymize_sentence(data, id_phrase):

    df = data[data['id_phrase'] == id_phrase]  

    if df['label'].nunique() > 1:

        df_lab = df[df['label'] != 'O']

        index = list(df_lab.index)
        lab_start = index[0]
        lab_end = index[-1]
        lab_amplitude = lab_end - lab_start

        if lab_amplitude < 50:

            wind_start = max(df.index[0], lab_start - 20)
            wind_end = min(df.index[-1], lab_end + 20)
            df_window = df.loc[wind_start : wind_end]

            proba = df_lab.shape[0] / lab_amplitude
            new_words = df_window['token'].copy()

            for row in range(wind_start, wind_end + 1):
                if df_window.loc[row].label == 'O':
                    dash = df_window.loc[row].token.startswith(chr(9601))
                    word = df_window.loc[row].token.replace(chr(9601), '')
                    synonym = synonymize_from_dic(word, cnrtl)
                    if synonym != 'No synonym found':
                        if dash:
                            new_words.loc[row] = chr(9601) + synonym
                        else:
                            new_words.loc[row] = synonym

            df_window['synonym'] = new_words
            
            return df_window
            
    return 'No synonymization available'